In [1]:
import pandas as pd
from mccn.client import MCCN
from tests.conftest import load_collection
from tests.utils import RASTER_FIXTURE_PATH

In [5]:
engine = MCCN(
    collection=load_collection(RASTER_FIXTURE_PATH / "dsm_config.json"),
    shape=100,
    bands=["red", "green", "blue", "dsm"],
    time_groupby="time",
)
ds = engine.load()
ds

<xarray.Dataset> Size: 282kB
Dimensions:      (time: 7, y: 100, x: 100)
Coordinates:
  * y            (y) float64 800B -35.05 -35.05 -35.05 ... -35.05 -35.05 -35.05
  * x            (x) float64 800B 147.3 147.3 147.3 147.3 ... 147.3 147.3 147.3
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 56B 2015-10-01T01:55:00 ... 2016-11-01...
Data variables:
    dsm          (time, y, x) float32 280kB 184.6 184.7 184.7 ... 186.4 186.4

In [ ]:
pd.DatetimeIndex(ds["time"].values).to_period("min").start_time